In [ ]:
from matpower import start_instance
from oct2py import octave

import numpy as np
import pandas as pd


def read(file_path: str, sheet_name: str):
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    return df.to_numpy()

file_path = 'ac_case25.xlsx'
BUS = read(file_path, 'bus')
GEN = read(file_path, 'generator')
TR = read(file_path, 'transformer')
LINE = read(file_path, 'branch')

In [2]:
print(BUS)

[[1 'PV' 108.0 22.0 1 1 0 1.05 0.95]
 [2 'PV' 97.0 20.0 1 1 0 1.05 0.95]
 [3 'PQ' 207.0 42.55 1 1 0 1.05 0.95]
 [4 'PQ' 74.0 15.0 1 1 0 1.05 0.95]
 [5 'PQ' 71.0 14.0 1 1 0 1.05 0.95]
 [6 'PQ' 136.0 28.0 2 1 0 1.05 0.95]
 [7 'PV' 125.0 25.0 2 1 0 1.05 0.95]
 [8 'PQ' 136.8 28.0 2 1 0 1.05 0.95]
 [9 'PQ' 175.0 36.0 1 1 0 1.05 0.95]
 [10 'PQ' 156.0 32.0 2 1 0 1.05 0.95]
 [11 'PQ' 0.0 0.0 3 1 0 1.05 0.95]
 [12 'PQ' 0.0 0.0 3 1 0 1.05 0.95]
 [13 'Swing' 265.0 54.0 3 1 0 1.05 0.95]
 [14 'PV' 194.0 39.0 3 1 0 1.05 0.95]
 [15 'PV' 253.6 51.2 4 1 0 1.05 0.95]
 [16 'PV' 115.0 23.0 4 1 0 1.05 0.95]
 [17 'PQ' 0.0 0.0 4 1 0 1.05 0.95]
 [18 'PV' 333.0 68.0 4 1 0 1.05 0.95]
 [19 'PQ' 208.15 42.55 3 1 0 1.05 0.95]
 [20 'PQ' 128.0 26.0 3 1 0 1.05 0.95]
 [21 'PV' 0.0 0.0 4 1 0 1.05 0.95]
 [22 'PV' 0.0 0.0 4 1 0 1.05 0.95]
 [23 'PV' 0.0 0.0 3 1 0 1.05 0.95]
 [24 'PQ' 0.0 0.0 4 1 0 1.05 0.95]
 [25 'PQ' 30.0 5.0 1 1 0 1.05 0.95]]


In [3]:
columns = ['BUS', 'Type', 'Pload(MW)', 'Qload(MVAR)', 'area', 'Vm(pu)', 'Va(degree)', 'maxVm', 'minVm']
BUS = pd.DataFrame(BUS, columns=columns)

BUS = pd.DataFrame(BUS)

# 'Type' 열에서 PQ, PV, Swing을 1, 2, 3으로 매핑
type_mapping = {'PQ': 1, 'PV': 2, 'Swing': 3}
BUS['Type'] = BUS['Type'].map(type_mapping)

zero_array = np.zeros(len(BUS))
BUS.insert(5, "Gs", zero_array)
BUS.insert(6, "Bs", zero_array)


col_name = BUS.columns[4]      # 이동할 열 이름
BUS.insert(6,                  # 삽입할 위치 (7번째, 0-based index 6)
           col_name,          # 열 이름
           BUS.pop(col_name)  # pop으로 꺼낸 열 데이터
          )

one_array=np.ones(len(BUS))
BUS.insert(9, "baseKV", one_array)
BUS.insert(10, "zone", one_array)

print(BUS)

   BUS  Type Pload(MW) Qload(MVAR)   Gs   Bs area Vm(pu) Va(degree)  baseKV  \
0    1     2     108.0        22.0  0.0  0.0    1      1          0     1.0   
1    2     2      97.0        20.0  0.0  0.0    1      1          0     1.0   
2    3     1     207.0       42.55  0.0  0.0    1      1          0     1.0   
3    4     1      74.0        15.0  0.0  0.0    1      1          0     1.0   
4    5     1      71.0        14.0  0.0  0.0    1      1          0     1.0   
5    6     1     136.0        28.0  0.0  0.0    2      1          0     1.0   
6    7     2     125.0        25.0  0.0  0.0    2      1          0     1.0   
7    8     1     136.8        28.0  0.0  0.0    2      1          0     1.0   
8    9     1     175.0        36.0  0.0  0.0    1      1          0     1.0   
9   10     1     156.0        32.0  0.0  0.0    2      1          0     1.0   
10  11     1       0.0         0.0  0.0  0.0    3      1          0     1.0   
11  12     1       0.0         0.0  0.0  0.0    3   

In [4]:
m = start_instance()

mpc = {
    'version': '2',
    'baseMVA': 100,
    'bus': BUS.astype(float),     # (n_bus × 13)
    'gen': GEN.astype(float),     # (n_gen × 21)
    'branch': LINE.astype(float), 
}    

In [5]:
mpc['bus'] = BUS.to_numpy().astype(float)
mpc['bus']

array([[  1.  ,   2.  , 108.  ,  22.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  2.  ,   2.  ,  97.  ,  20.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  3.  ,   1.  , 207.  ,  42.55,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  4.  ,   1.  ,  74.  ,  15.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  5.  ,   1.  ,  71.  ,  14.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  6.  ,   1.  , 136.  ,  28.  ,   0.  ,   0.  ,   2.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  7.  ,   2.  , 125.  ,  25.  ,   0.  ,   0.  ,   2.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  8.  ,   1.  , 136.8 ,  28.  ,   0.  ,   0.  ,   2.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  9.  ,   1.  , 175.  , 

In [6]:
print(GEN)

[[  1.      1.     10.      0.     10.      0.      1.035 100.      1.   ]
 [  2.      1.     10.      0.     10.      0.      1.035 100.      1.   ]
 [  3.      1.     76.      0.     30.    -25.      1.035 100.      1.   ]
 [  4.      1.     76.      0.     30.    -25.      1.035 100.      1.   ]
 [  5.      2.     10.      0.     10.      0.      1.035 100.      1.   ]
 [  6.      2.     10.      0.     10.      0.      1.035 100.      1.   ]
 [  7.      2.     76.      0.     30.    -25.      1.035 100.      1.   ]
 [  8.      2.     76.      0.     30.    -25.      1.035 100.      1.   ]
 [  9.      7.     80.      0.     60.      0.      1.025 100.      1.   ]
 [ 10.      7.     80.      0.     60.      0.      1.025 100.      1.   ]
 [ 11.      7.     80.      0.     60.      0.      1.025 100.      1.   ]
 [ 12.     13.     95.1     0.     80.      0.      1.02  100.      1.   ]
 [ 13.     13.     95.1     0.     80.      0.      1.02  100.      1.   ]
 [ 14.     13.     95.1  

In [ ]:
columns = ['Id','Bus','PG(MW)','QG(MVAR)','QMAX(MVAR)','QMIN(MVAR)','Voltage setpoint(pu)','MBASE(MW)','STATUS']
GEN = pd.DataFrame(GEN, columns=columns)

GEN = pd.DataFrame(GEN)

GEN = GEN.drop(columns=['Id'])

zero_array = np.zeros(len(GEN))
GEN.insert(8, "Pmax", zero_array)
GEN.insert(9, "Pmin", zero_array)
GEN.insert(10, "Pc1", zero_array)
GEN.insert(11, "Pc2", zero_array)
GEN.insert(12, "Qc1min", zero_array)
GEN.insert(13, "Qc1max", zero_array)
GEN.insert(14, "Qc2min", zero_array)
GEN.insert(15, "Qc2max", zero_array)
GEN.insert(16, "ramp rate for load following/AGC (MW/min)", zero_array)
GEN.insert(17, "ramp rate for 10 minute reserves (MW)", zero_array)
GEN.insert(18, "ramp rate for 30 minute reserves (MW)", zero_array)
GEN.insert(19, "ramp rate for reactive power (2 sec timescale) (MVAr/min)", zero_array)
GEN.insert(20, "APF, area participation factor", zero_array)

print(GEN)

     Bus  PG(MW)  QG(MVAR)  QMAX(MVAR)  QMIN(MVAR)  Voltage setpoint(pu)  \
0    1.0    10.0       0.0        10.0         0.0                 1.035   
1    1.0    10.0       0.0        10.0         0.0                 1.035   
2    1.0    76.0       0.0        30.0       -25.0                 1.035   
3    1.0    76.0       0.0        30.0       -25.0                 1.035   
4    2.0    10.0       0.0        10.0         0.0                 1.035   
5    2.0    10.0       0.0        10.0         0.0                 1.035   
6    2.0    76.0       0.0        30.0       -25.0                 1.035   
7    2.0    76.0       0.0        30.0       -25.0                 1.035   
8    7.0    80.0       0.0        60.0         0.0                 1.025   
9    7.0    80.0       0.0        60.0         0.0                 1.025   
10   7.0    80.0       0.0        60.0         0.0                 1.025   
11  13.0    95.1       0.0        80.0         0.0                 1.020   
12  13.0    

In [8]:
mpc['gen'] = GEN.to_numpy().astype(float)
mpc['gen']

array([[  1.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  2.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
  

In [9]:
print(LINE)

[[1.0000e+00 2.0000e+00 2.6000e-03 1.3900e-02 4.6110e-01]
 [1.0000e+00 3.0000e+00 5.4600e-02 2.1120e-01 5.7200e-02]
 [1.0000e+00 5.0000e+00 2.1800e-02 8.4500e-02 2.2900e-02]
 [2.0000e+00 4.0000e+00 3.2800e-02 1.2670e-01 3.4300e-02]
 [2.0000e+00 6.0000e+00 4.9700e-02 1.9200e-01 5.2000e-02]
 [3.0000e+00 9.0000e+00 3.0800e-02 1.1900e-01 3.2200e-02]
 [3.0000e+00 2.4000e+01 2.3000e-03 8.3900e-02 0.0000e+00]
 [4.0000e+00 9.0000e+00 2.6800e-02 1.0370e-01 2.8100e-02]
 [5.0000e+00 1.0000e+01 2.2800e-02 8.8300e-02 2.3900e-02]
 [6.0000e+00 1.0000e+01 1.3900e-02 6.0500e-02 2.4590e+00]
 [7.0000e+00 8.0000e+00 1.5900e-02 6.1400e-02 1.6600e-02]
 [8.0000e+00 9.0000e+00 4.2700e-02 1.6510e-01 4.4700e-02]
 [8.0000e+00 1.0000e+01 4.2700e-02 1.6510e-01 4.4700e-02]
 [9.0000e+00 1.1000e+01 2.3000e-03 8.3900e-02 0.0000e+00]
 [9.0000e+00 1.2000e+01 2.3000e-03 8.3900e-02 0.0000e+00]
 [1.0000e+01 1.1000e+01 2.3000e-03 8.3900e-02 0.0000e+00]
 [1.0000e+01 1.2000e+01 2.3000e-03 8.3900e-02 0.0000e+00]
 [1.1000e+01 1

In [10]:
columns = ['f','t','tap']
TR = pd.DataFrame(TR, columns=columns)

TR = pd.DataFrame(TR)

TR['tap'] = 1 / TR['tap']

print(TR)

      f     t       tap
0   3.0  24.0  0.970874
1   9.0  11.0  0.970874
2   9.0  12.0  0.970874
3  10.0  11.0  0.980392
4  10.0  12.0  0.980392


In [11]:
columns = ['f','t','r','x','b']
LINE = pd.DataFrame(LINE, columns=columns)

LINE = pd.DataFrame(LINE)

zero_array = np.zeros(len(LINE))
LINE.insert(5, "rateA", zero_array)
LINE.insert(6, "rateB", zero_array)
LINE.insert(7, "rateC", zero_array)
LINE.insert(8, "tap", zero_array)
LINE.insert(9, "shift", zero_array)
LINE.insert(10, "initial branch status", np.ones(len(LINE)))
LINE.insert(11, "minimum angle difference", zero_array)
LINE.insert(12, "maximum angle difference", zero_array)

LINE = LINE.merge(TR, on=['f', 't'], how='left', suffixes=('', '_new'))

# 2) `tap_new`가 존재하는 경우만 tap 값을 업데이트
LINE['tap'] = LINE['tap_new'].fillna(LINE['tap'])

# 3) 중간 컬럼 제거
LINE = LINE.drop(columns=['tap_new'])

print(LINE)

       f     t        r         x       b  rateA  rateB  rateC       tap  \
0    1.0   2.0  0.00260  0.013900  0.4611    0.0    0.0    0.0  0.000000   
1    1.0   3.0  0.05460  0.211200  0.0572    0.0    0.0    0.0  0.000000   
2    1.0   5.0  0.02180  0.084500  0.0229    0.0    0.0    0.0  0.000000   
3    2.0   4.0  0.03280  0.126700  0.0343    0.0    0.0    0.0  0.000000   
4    2.0   6.0  0.04970  0.192000  0.0520    0.0    0.0    0.0  0.000000   
5    3.0   9.0  0.03080  0.119000  0.0322    0.0    0.0    0.0  0.000000   
6    3.0  24.0  0.00230  0.083900  0.0000    0.0    0.0    0.0  0.970874   
7    4.0   9.0  0.02680  0.103700  0.0281    0.0    0.0    0.0  0.000000   
8    5.0  10.0  0.02280  0.088300  0.0239    0.0    0.0    0.0  0.000000   
9    6.0  10.0  0.01390  0.060500  2.4590    0.0    0.0    0.0  0.000000   
10   7.0   8.0  0.01590  0.061400  0.0166    0.0    0.0    0.0  0.000000   
11   8.0   9.0  0.04270  0.165100  0.0447    0.0    0.0    0.0  0.000000   
12   8.0  10

In [12]:
mpc['branch'] = LINE.to_numpy().astype(float)
mpc['branch']

array([[1.00000000e+00, 2.00000000e+00, 2.60000000e-03, 1.39000000e-02,
        4.61100000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.00000000e+00, 3.00000000e+00, 5.46000000e-02, 2.11200000e-01,
        5.72000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.00000000e+00, 5.00000000e+00, 2.18000000e-02, 8.45000000e-02,
        2.29000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.00000000e+00, 4.00000000e+00, 3.28000000e-02, 1.26700000e-01,
        3.43000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.00000000e+00, 6.00000000e+

In [13]:
mpopt = m.mpoption('verbose', 2)

results = m.runpf(mpc, mpopt)


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

 it    max residual        max ∆x
----  --------------  --------------
  0      5.866e+00           -
  1      4.713e-01       4.291e-01
  2      8.696e-03       2.581e-02
  3      7.973e-06       7.612e-04
  4      1.057e-11       8.191e-07
Newton's method converged in 4 iterations.
PF successful

Converged in 0.34 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             25     Total Gen Capacity       0.0        -535.0 to 1776.0
Generators        33     On-line Capacity         0.0        -535.0 to 1776.0
Committed Gens    33     Generation (actual)   2865.9             464.3
Loads             18     Load                  2812.5             571.3
  Fixed           18       Fixed               2812.5        